In [5]:
import numpy as np
import dipy
from dipy.core.gradients import gradient_table
from dipy.viz import actor, window, ui
from dipy.viz import fvtk
from nibabel.streamlines import load as load_trk
from nibabel.streamlines import Tractogram
from dipy.tracking.streamline import Streamlines
from dipy.tracking import utils
from dipy.viz.colormap import line_colors
from dipy.segment.clustering import QuickBundles
from dipy.tracking.streamline import streamline_near_roi
from dipy.tracking.life import transform_streamlines
import nibabel as nib
from dipy.align.streamlinear import whole_brain_slr, slr_with_qb
from dipy.segment.bundles import RecoBundles
from dipy.io.image import load_nifti, save_nifti
import os
import glob 
from nibabel.streamlines import Field
from nibabel.orientations import aff2axcodes
%matplotlib

Using matplotlib backend: Qt5Agg


In [6]:
def show_streamlines(streamlines):
    from dipy.viz import window, actor
    ren = window.Renderer()
    ren.add(actor.line(streamlines))
    #window.record(ren, n_frames=10, out_path='masked_after.png', size=(600, 600))
    window.show(ren)

In [11]:
def show_streamlines_two(streamlines, str1):
    from dipy.viz import window, actor
    ren = window.Renderer()
    ren.add(actor.line(streamlines, colors=(1,1,1)))
    ren.add(actor.line(str1))
    #window.record(ren, n_frames=10, out_path='masked_after.png', size=(600, 600))
    window.show(ren)

In [7]:
def save_trk_n(fname, streamlines, affine, vox_size=None, shape=None, header=None):
    """ function Helper for saving trk files.

    Parameters
    ----------
    fname : str
        output trk filename
    streamlines : list of 2D arrays
        Each 2D array represents a sequence of 3D points (points, 3).
    affine : array_like (4, 4)
        The mapping from voxel coordinates to streamline points.
    vox_size : array_like (3,)
        The sizes of the voxels in the reference image.
    shape : array, shape (dim,)
        The shape of the reference image.
    header : dict
        header from a trk file

    """
    if vox_size and shape:
        if not isinstance(header, dict):
            header = {}
        header[Field.VOXEL_TO_RASMM] = affine.copy()
        header[Field.VOXEL_SIZES] = vox_size
        header[Field.DIMENSIONS] = shape
        header[Field.VOXEL_ORDER] = "".join(aff2axcodes(affine))

    tractogram = nib.streamlines.Tractogram(streamlines)
    tractogram.affine_to_rasmm = affine
    trk_file = nib.streamlines.TrkFile(tractogram, header=header)
    nib.streamlines.save(trk_file, fname)


def load_trk_(filename):
    """ function Helper for Loading trk files.

    Parameters
    ----------
    filename : str
        input trk filename

    Returns
    -------
    streamlines : list of 2D arrays
        Each 2D array represents a sequence of 3D points (points, 3).
    hdr : dict
        header from a trk file

    """
    trk_file = nib.streamlines.load(filename)
    return trk_file.streamlines, trk_file.header

In [8]:
os.chdir('/home/bramsh/Desktop/scratch/MNI_space/whole_brain')
brain=load_trk("whole_brain_MNI.trk")
whole_brain=brain.streamlines

In [9]:
#os.chdir('/home/bramsh/Desktop/scratch/MNI_space/CP_data/3106/3106_second')
#mov_file = load_trk("3106_second_streamlines.trk")

mov_file = load_trk("/home/bramsh/Desktop/scratch/3104_first_streamlines_small.trk")
moving = mov_file.streamlines

In [10]:
mov_slr , affine ,_ ,_ = whole_brain_slr(whole_brain, moving)

Progressive Registration is Enabled
 Translation  (3 parameters)...
 Rigid  (6 parameters) ...
 Similarity (7 parameters) ...
 Scaling (9 parameters) ...
 Affine (12 parameters) ...


In [12]:
show_streamlines_two(whole_brain, mov_slr)

/home/bramsh/dipy/dipy/viz/colormap.py:233: RuntimeWarning: invalid value encountered in true_divide
  orient = np.abs(orient / np.linalg.norm(orient))


In [8]:
os.chdir('/home/bramsh/Desktop/scratch/MNI_space/CP_data/3106/3106_second')
file = "denoised_3106_second.nii.gz"
img = nib.load(file)
data, data_affine = load_nifti(file)
volume= data.shape[:3]
voxel= img.header.get_zooms()[:3]
save_trk_n(  file[9:len(file)-7]+'_to_MNI_space.trk', mov_slr, affine=data_affine, vox_size=voxel, shape=volume, header=img.header)

In [7]:
#try
'''os.chdir('/home/bramsh/Desktop/scratch/MNI_space/CP_data/3104_first')
file = '3104_first_to_MNI_space.trk'
mov_file = load_trk(file)
mov_stream = mov_file.streamlines'''

In [15]:
show_streamlines(mov_stream)

/home/bramsh/dipy/dipy/viz/colormap.py:233: RuntimeWarning: invalid value encountered in true_divide
  orient = np.abs(orient / np.linalg.norm(orient))


In [9]:
mov_stream = Streamlines(mov_slr)

In [10]:
my_tract = RecoBundles(mov_stream )

# Cluster streamlines using QBx
 Tractogram has 1969260 streamlines
 Size is 3654.752 MB
 Distance threshold 15.000
 Resampled to 20 points
 Size is 901.456 MB
 Duration of resampling is 1.746 sec.
 QBX phase starting...
 Merging phase starting ...
 QuickBundlesX time for 1969260 random streamlines
 Duration 73.179 sec. 

 Streamlines have 4137 centroids
 Total duration 74.927 sec. 



In [11]:
#load files
os.chdir("/home/bramsh/Desktop/scratch/MNI_space/bundles")
all_files = list(glob.glob('*.trk'))                                      
print(all_files)

['ICP_R.trk', 'CNII_R.trk', 'ILF_L.trk', 'CS_R.trk', 'CNIII_L.trk', 'RST_L.trk', 'UF_L.trk', 'EMC_R.trk', 'PC.trk', 'CST_R.trk', 'CNVII_L.trk', 'ILF_R.trk', 'DLF_R.trk', 'STT_R.trk', 'OPT_R.trk', 'U_R.trk', 'F_R.trk', 'CT_R.trk', 'CTT_L.trk', 'SLF_L.trk', 'OPT_L.trk', 'VOF_R.trk', 'UF_R.trk', 'MLF_L.trk', 'FPT_L.trk', 'TPT_L.trk', 'U_L.trk', 'AC.trk', 'CNVIII_R.trk', 'AST_L.trk', 'CB_R.trk', 'CNII_L.trk', 'CNVIII_L.trk', 'EMC_L.trk', 'OR_L.trk', 'MLF_R.trk', 'CNIV_R.trk', 'IFOF_R.trk', 'MCP.trk', 'STT_L.trk', 'CST_L.trk', 'ML_R.trk', 'CC.trk', 'CS_L.trk', 'RST_R.trk', 'MdLF_R.trk', 'DLF_L.trk', 'PPT_R.trk', 'CNVII_R.trk', 'TPT_R.trk', 'V.trk', 'C_L.trk', 'AR_R.trk', 'IFOF_L.trk', 'SCP.trk', 'CTT_R.trk', 'ML_L.trk', 'PPT_L.trk', 'MdLF_L.trk', 'OR_R.trk', 'LL_L.trk', 'AST_R.trk', 'ICP_L.trk', 'CB_L.trk', 'AR_L.trk', 'AF_L.trk', 'AF_L_1.trk', 'CNV_R.trk', 'CNIII_R.trk', 'C_R.trk', 'CT_L.trk', 'FPT_R.trk', 'SLF_R.trk', 'LL_R.trk', 'F_L.trk', 'CNV_L.trk', 'AF_R.trk', 'CNIV_L.trk', 'VOF_L.tr

In [12]:
file = "3106_second"
all_files = ['AF_R.trk', 'AF_L.trk', 'CST_R.trk', 'CST_L.trk']

In [13]:
for i in range(len(all_files)):
    b=load_trk(all_files[i])
    bundle=b.streamlines
    reco_bundle, labels, pruned = my_tract.recognize(bundle, model_clust_thr=5)
    #an other slr
    #show_streamlines(reco_bundle)
    
    '''slr_reco_bundle, slr_affine, _, _ = slr_with_qb(bundle, pruned, "affine", rm_small_clusters=2, 
                          greater_than=0, less_than=np.Inf, qb_thr=0.5)'''
    
    #original affine for saving????
    
    
    os.chdir("/home/bramsh/Desktop/scratch/MNI_space/CP_data/3106/3106_second/bundles")
    print("********* saving trk file ************")
    print(all_files[i])
    save_trk_n(file+'_'+all_files[i][0:len(all_files[i])], pruned, affine=np.eye(4), vox_size=voxel, shape=volume)
    print("********* saving labels file ************")
    np.save(file+'_'+all_files[i][0:len(all_files[i])-4]+"_labels", labels)
    os.chdir("/home/bramsh/Desktop/scratch/MNI_space/bundles")

## Recognize given bundle ## 

# Cluster model bundle using QBx
 Model bundle has 1145 streamlines
 Distance threshold 5.000
 Resampled to 20 points
 Size is 0.262 MB
 Duration of resampling is 0.002 sec.
 QBX phase starting...
 Merging phase starting ...
 QuickBundlesX time for 500000 random streamlines
 Duration 0.014 sec. 

 Model bundle has 5 centroids
 Duration 0.016 sec. 

# Reduce search space
 Reduction threshold 20.000
 Reduction distance mdf
 Using MDF
 Number of neighbor streamlines 16190
 Duration 0.098 sec. 

# Local SLR of neighb_streamlines to model
 Square-root of BMD is 12.106
 Number of iterations 49
 Matrix size (400, 600)
[[  0.954   0.122   0.295  -4.149]
 [ -0.044   0.972  -0.258  -1.75 ]
 [ -0.316   0.231   0.927  22.219]
 [  0.      0.      0.      1.   ]]
[ -0.054  -9.751   7.016  14.019  18.324  -2.657   1.006]
 Duration 17.910 sec. 

# Prune streamlines using the MDF distance
 Pruning threshold 10.000
 Pruning distance mdf
 Resampled to 20 points
 Size is 7.4

In [1]:
0

0

In [13]:
show_streamlines(pruned)

In [14]:
show_streamlines(reco_bundle)

In [15]:
def show_streamlines_two(streamlines, str1):
    from dipy.viz import window, actor
    ren = window.Renderer()
    ren.add(actor.line(streamlines, colors=(1,1,1)))
    ren.add(actor.line(str1))
    #window.record(ren, n_frames=10, out_path='masked_after.png', size=(600, 600))
    window.show(ren)

In [17]:
show_streamlines_two(pruned, reco_bundle)

In [37]:
slr_reco_bundle, slr_affine, _, _ = slr_with_qb(bundle, pruned, "affine", rm_small_clusters=2, 
                          greater_than=0, less_than=np.Inf, qb_thr=0.5)

Progressive Registration is Enabled
 Translation  (3 parameters)...
 Rigid  (6 parameters) ...
 Similarity (7 parameters) ...
 Scaling (9 parameters) ...
 Affine (12 parameters) ...


In [31]:
os.chdir('/home/bramsh/Desktop/scratch/MNI_space/p_data/3105_first/bundles')
file = '3105_first_CST_L.trk'
org=load_trk(file)
streamlines_org=org.streamlines


In [32]:
show_streamlines_two(streamlines_org, bundle)

In [11]:
slr_with_qb?